# Caso Práctico: Rescate del Chatbot de Clasificación de "MediQuery" (Diagnóstico de Overfitting)

**Empresa MediQuery Contexto**: "MediQuery" es una startup de telemedicina. Han lanzado un chatbot para clasificar las consultas entrantes de los pacientes y dirigirlas al departamento correcto. Las categorías son: [Cita_Medica], [Facturacion], [Consulta_Resultados] y [Urgencia_Medica].

El Problema (¡Crisis Real!): El equipo junior entrenó un modelo bert-base-uncased usando un dataset pequeño de 1,000 ejemplos etiquetados a mano. En sus notebooks, el modelo alcanzó un 99.8% de accuracy en el conjunto de entrenamiento y un 98% en el de validación.

Sin embargo, una semana después de lanzarlo a producción, el sistema colapsó. El modelo está clasificando consultas como "Me duele el pecho y no puedo respirar" en la categoría [Facturacion]. El modelo no ha generalizado; ha memorizado los 1,000 ejemplos de entrenamiento. Esto es un caso clásico de sobreajuste (overfitting).

**El Desafío:** Tú eres el/la Ingeniero/a de ML que han contratado para arreglar el desastre. Tu objetivo no es (solo) entrenar un modelo, sino diagnosticar, solucionar y demostrar cómo evitar que el sobreajuste destruya la fiabilidad del sistema.

## Enunciado del Problema y Tarea

Tu objetivo es replicar el error del equipo junior y luego implementar dos estrategias de regularización/optimización de Transformers para rescatar el modelo y hacerlo generalizable.

Conjunto de Datos: Se te proporciona un archivo mediquery_1k_samples.csv. Es un dataset intencionalmente pequeño y "fácil" de memorizar.

- id_consulta: Identificador único.

- texto_consulta: El texto del paciente.

- etiqueta: (Facturacion, Cita_Medica, Consulta_Resultados, Urgencia_Medica)

## Pasos Obligatorios:

1. Carga y Preparación de Datos:

- Cargar el mediquery_1k_samples.csv.

- Convertir las etiquetas de texto a IDs numéricos.

- Dividir los datos (ej. 80% entrenamiento, 20% validación).

2. Paso 1: Replicar el Desastre (El Modelo Sobreajustado)

- Cargar el tokenizador y el modelo "grande" (bert-base-uncased).

- Configurar los TrainingArguments para forzar el sobreajuste:

*  Entrenar durante un número alto de épocas (ej. num_train_epochs=50).

*  Usar un learning_rate estándar (ej. 5e-5).

- Entrenar el modelo.

- Entregable de Diagnóstico: Generar un gráfico (usando matplotlib o similar) que muestre la training_loss vs. la validation_loss por época. Debes mostrar visualmente cómo la training_loss sigue bajando mientras la validation_loss se estanca o empieza a subir.


3. Paso 2: Rescate - Estrategia A (Regularización del Entrenamiento)

- Hipótesis: El modelo es bueno, pero entrenó demasiado tiempo.

- antener el modelo bert-base-uncased.

- Modificar los TrainingArguments para implementar Early Stopping:

 * evaluation_strategy = "epoch"

 * save_strategy = "epoch"

 * load_best_model_at_end = True (Esto es crucial: el Trainer recargará las pesas del mejor epoch, no del último).

- Re-entrenar el modelo. El Trainer debería detenerse automáticamente (ej. en la época 3 o 4) cuando la validation_loss deje de mejorar.

4. Paso 3: Rescate - Estrategia B (Reducción de Arquitectura)

- Hipótesis: El modelo (bert-base) es demasiado grande y potente para un dataset tan pequeño. Es como usar un motor de Fórmula 1 para ir a comprar el pan: memorizará el camino.

- Cambiar la arquitectura del modelo. En lugar de bert-base-uncased, carga un modelo mucho más ligero, como prajjwal1/bert-tiny o google/bert_uncased_L-4_H-256_A-4 (un BERT más pequeño).

- Nota: Asegúrate de usar el mismo tokenizador (bert-base-uncased) si el modelo ligero lo comparte, o carga el tokenizador específico del modelo ligero.

- Entrena este modelo ligero (puedes usar el Early Stopping del Paso 2).

5. Paso 4: Análisis y Conclusión:

- Crear una función de inferencia clasificar_consulta(texto) que funcione con tu modelo rescatado.

- Probar el modelo con frases nuevas (no vistas en el CSV) que sean ambiguas o críticas:

Ej 1 (Crítico): "Me duele el pecho y me cuesta respirar."

Ej 2 (Facturación): "¿Por qué el cargo de mi última visita es tan alto?"

Ej 3 (Ambiguo): "No sé qué hacer con los resultados que me dieron."

- Redactar/comentamos una breve conclusión (en Markdown) explicando por qué la Estrategia A o B es la mejor solución para "MediQuery" en términos de coste, velocidad y fiabilidad.